# Polytech SI3 - Données numériques: classification de sons
Diane Lingrand
2022-2023


In [36]:
#chargement des librairies
import matplotlib.pyplot as plt
import numpy as np 
import librosa
import librosa.display
import IPython.display as ipd

In [37]:
import glob

In [38]:
basedir="/home/tsukoyachi/Documents/Ecole/SI3/ComputerScience/TD/S6/Données Numériques/TD4/"
# à adapter 
#attention, sous windows, il faut changer en "C:\\Users\\monNom\\monCoursPrefere\\"
classes = ["cat","dog","bird"] 
nbClasses = len(classes)

for cl in classes:
    listSons = glob.glob(basedir+cl+"/*.wav")
    print(cl,len(listSons))

cat 1733
dog 1746
bird 1731


## algorithme kNN

In [39]:
#knn algo
def voteKnn(xtrain, ytrain, nKnn, newData):
    
    classWhenAmbiguity = 2

    d = []
    for PX in xtrain:
        d.append(np.sqrt(np.power(PX[0] - newData[0], 2) + np.power(PX[1] - newData[1], 2) + np.power(PX[2] - newData[2], 2)))

    r = np.argsort(d)

    # nKNN smallest distances indices
    nei1 = r[0:nKnn]

    # nKNN closest data
    Xn1 = []
    yn1 = []
    for ji in nei1:
        Xn1.append(xtrain[ji])
        yn1.append(ytrain[ji])

    cl, co = np.unique(yn1, return_counts=True)
    nbOfMC = np.count_nonzero(co == np.max(co))  # Permet de récupérer le nombre de classe qui ont le nombre d'occurrence max du tableau
    if nbOfMC > 1:
        classe1 = classWhenAmbiguity
    else:
        maxOccIndex1 = np.argmax(co)  # renvoie l'indice de l'occurrence la plus élevée
        classe1 = cl[maxOccIndex1]  # on récupère la classe correspondant à cette occurrence la plus élevée, autrement dit on récupère la classe ayant le nombre d'occurrence le plus élevé, grâce à leur indice identique (dans les tableaux classes et counts, une classe à un indice précis va trouver son occurrence au même indice dans le tableau counts)
    return classe1


## Représentation des sons par 1 MFCC

In [40]:
# loading train dataset
nb=200 # for each class
i = 0
clNumber = 0
yTrain = []
Xtrain = np.empty(shape=(nb*nbClasses, 13), dtype=float)
for cl in classes:
    listSons = glob.glob(basedir+cl+"/*.wav")
    for s in listSons[:nb]:
        (sig,rate) = librosa.load(s)
        mfcc_feat = librosa.feature.mfcc(y=sig,sr=rate,n_mfcc=13, hop_length=len(sig)+1)
        Xtrain[i] = mfcc_feat.reshape(13)
        i += 1
    yTrain += [clNumber]*nb
    clNumber += 1

yTrain = np.array(yTrain)

In [41]:
## loading test dataset
i = 0
clNumber = 0
yTest = []
Xtest = np.empty(shape=(nb*nbClasses, 13), dtype=float)
for cl in classes:
    listSons = glob.glob(basedir+cl+"/*.wav")
    for s in listSons[-nb:]:
    #print("###",s,"###")
        (sig,rate) = librosa.load(s)
        mfcc_feat = librosa.feature.mfcc(y=sig,sr=rate,n_mfcc=13, hop_length=len(sig)+1)    
        Xtest[i] = mfcc_feat.reshape(13)
        i += 1
    yTest += [clNumber]*nb
    clNumber += 1

yTest = np.array(yTest)

## Représentation des sons par la moyenne des MFCCs (longueur standard)

In [42]:
# loading train dataset
nb=200 # for each class
i = 0
clNumber = 0
yTrain = []
Xtrain = np.empty(shape=(nb*nbClasses, 13), dtype=float)
for cl in classes:
    listSons = glob.glob(basedir+cl+"/*.wav")
    for s in listSons[:nb]:
        (sig,rate) = librosa.load(s)
        mfcc_feat = librosa.feature.mfcc(y=sig,sr=rate,n_mfcc=13)
        Xtrain[i] = np.mean(mfcc_feat, axis=1)
        i += 1
    yTrain += [clNumber]*nb
    clNumber += 1

yTrain = np.array(yTrain)

In [43]:
## loading test dataset
i = 0
clNumber = 0
yTest = []
Xtest = np.empty(shape=(nb*nbClasses, 13), dtype=float)
for cl in classes:
    listSons = glob.glob(basedir+cl+"/*.wav")
    for s in listSons[-nb:]:
    #print("###",s,"###")
        (sig,rate) = librosa.load(s)
        mfcc_feat = librosa.feature.mfcc(y=sig,sr=rate,n_mfcc=13)
        Xtest[i] = np.mean(mfcc_feat, axis=1)
        i += 1
    yTest += [clNumber]*nb
    clNumber += 1

yTest = np.array(yTest)

## Classification par kNN

**Question 1**: Pour chaque représentation des fichiers sons, calculez les prédictions d'une classification par kNN. Pour cela, il faudra tester plusieurs valeurs de k.

In [44]:
numberNN = 5

In [45]:
#predictions
predTrain1 = [voteKnn(Xtrain, yTrain, numberNN, i) for i in Xtrain] # tableau des résultats de knn avec xTrain1
predTrain2 = [voteKnn(Xtrain, yTrain, numberNN, i) for i in Xtrain] # tableau des résultats de knn avec xTrain2

predTest1 = [voteKnn(Xtrain, yTrain, numberNN, i) for i in Xtest]  # tableau des résultats de knn avec xTest1
predTest2 = [voteKnn(Xtrain, yTrain, numberNN, i) for i in Xtest]  # tableau des résultats de knn avec xTest2

print(predTrain1)
print(predTrain2)
print(predTest1)
print(predTest2)

[1, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 1, 0, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 1, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 2, 0, 2, 2, 2, 1, 2, 0, 2, 0, 2, 1, 2, 0, 0, 0, 0, 2, 0, 0, 1, 0, 2, 1, 0, 2, 0, 0, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 1, 0, 0, 0, 0, 2, 2, 0, 0, 2, 1, 0, 1, 0, 0, 0, 2, 2, 2, 1, 2, 2, 1, 0, 2, 0, 0, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 0, 2, 0, 2, 2, 2, 1, 1, 0, 2, 2, 2, 0, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 0, 2, 2, 2, 2, 1, 2, 1, 0, 1, 0, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 0, 1, 0, 2, 2, 1, 1, 2, 2, 2, 0, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 0, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 0, 

**Question 2**: Calculez les matrices de confusion, les métriques de précision 'accuracy' et de score F1 pour les données d'apprentissage et de test. Pour cela, écrire 3 fonctions: <ul> <li> une fonction qui affiche une matrice de confusion</li> <li>une fonction qui calcule la précision (accuracy)</li><li>une fonction qui calcule le score F1 d'une classe</li></ul>

In [46]:
def matriceConfusion(predTrain, nbClasses, nbSoundEachClass):
    matriceConfusion = np.zeros((nbClasses, nbClasses))

    for cl in range(nbClasses):
        for i in range(nbSoundEachClass):
            pred = predTrain[cl * nbSoundEachClass + i]
            matriceConfusion[cl][pred] += 1
    return matriceConfusion

confusionMatrixTrain1 = matriceConfusion(predTrain, nbClasses, nb)
confusionMatrixTrain2 = matriceConfusion(predTrain, nbClasses, nb)
confusionMatrixTest1 = matriceConfusion(predTest, nbClasses, nb)
confusionMatrixTest2 = matriceConfusion(predTest, nbClasses, nb)

print("numberNN =", numberNN)
print("xTrain1 dans xTrain1 :\n", confusionMatrixTrain1)
print("xTrain2 dans xTrain2 :\n", confusionMatrixTrain2)
print("xTest1 dans xTrain1 :\n", confusionMatrixTest1)
print("xTest2 dans xTrain2 :\n", confusionMatrixTest2)


def selections(matriceConfusion, classeCible):
    vp = 0
    vn = 0
    fp = 0
    fn = 0
    nbClasses = len(matriceConfusion)

    for i in range(nbClasses):
        for j in range(nbClasses):
            if (i == classeCible):
                if (j == classeCible):
                    vp += matriceConfusion[i][j]
                else:
                    fn += matriceConfusion[i][j]
            else:
                if (j == classeCible):
                    fp += matriceConfusion[i][j]
                else:
                    vn += matriceConfusion[i][j]
    return vp, vn, fp, fn

def totaleAccuracy(matriceConfusion):
    vp, vn, fp, fn = (0, 0, 0, 0)
    for i in range(len(matriceConfusion)):
        vpTemp, vnTemp, fpTemp, fnTemp = selections(matriceConfusion, i)
        vp += vpTemp
        vn += vnTemp
        fp += fpTemp
        fn += fnTemp
    return (vp + vn) / (vp + vn + fp + fn)

print("Accuracy :")
print("xTrain1 dans xTrain1 :", totaleAccuracy(confusionMatrixTrain1))
print("xTrain2 dans xTrain2 :", totaleAccuracy(confusionMatrixTrain2))
print("xTest1 dans xTrain1 :", totaleAccuracy(confusionMatrixTest1))
print("xTest2 dans xTrain2 :", totaleAccuracy(confusionMatrixTest2))


def scoref1(matriceConfusion):
    vp, vn, fp, fn = (0, 0, 0, 0)
    for i in range(len(matriceConfusion)):
        vpTemp, vnTemp, fpTemp, fnTemp = selections(matriceConfusion, i)
        vp += vpTemp
        vn += vnTemp
        fp += fpTemp
        fn += fnTemp
    return 2 * vp / (2 * vp + fp + fn)

print("Score f1 :")
print("xTrain1 dans xTrain1 :", scoref1(confusionMatrixTrain1))
print("xTrain2 dans xTrain2 :", scoref1(confusionMatrixTrain2))
print("xTest1 dans xTrain1 :", scoref1(confusionMatrixTest1))
print("xTest2 dans xTrain2 :", scoref1(confusionMatrixTest2))

numberNN = 5
xTrain1 dans xTrain1 :
 [[112.  16.  72.]
 [ 26.  78.  96.]
 [ 10.  34. 156.]]
xTrain2 dans xTrain2 :
 [[112.  16.  72.]
 [ 26.  78.  96.]
 [ 10.  34. 156.]]
xTest1 dans xTrain1 :
 [[ 79.  28.  93.]
 [ 34.  55. 111.]
 [ 28.  45. 127.]]
xTest2 dans xTrain2 :
 [[ 79.  28.  93.]
 [ 34.  55. 111.]
 [ 28.  45. 127.]]
Accuracy :
xTrain1 dans xTrain1 : 0.7177777777777777
xTrain2 dans xTrain2 : 0.7177777777777777
xTest1 dans xTrain1 : 0.6233333333333333
xTest2 dans xTrain2 : 0.6233333333333333
Score f1 :
xTrain1 dans xTrain1 : 0.5766666666666667
xTrain2 dans xTrain2 : 0.5766666666666667
xTest1 dans xTrain1 : 0.435
xTest2 dans xTrain2 : 0.435


**Question 3:** Affichez la courbe de la précision (accuracy) en fonction du k de kNN pour les données d'apprentissage et pour les données de test.

In [ ]:
def metriques(xTrain, yTrain, newValues, numberNN, classes, nbSoundEachClass):
    nbClasses = len(classes)

    predNew = [voteKnn(xTrain, yTrain, numberNN, i) for i in newValues]
    confusionMatrix = matriceConfusion(predNew, nbClasses, nbSoundEachClass)
    accuracy = totaleAccuracy(confusionMatrix)
    f1 = scoref1(confusionMatrix)
    return accuracy, f1


resTrain2 = []
resTest2 = []

for i in range(1, 200):
    resTrain2.append(metriques(Xtrain, yTrain, Xtrain, i, classes, nb))
    resTest2.append(metriques(Xtrain, yTrain, Xtest, i, classes, nb))

resInit = [i for i in range(1, 200)]

resTrain2 = np.array(resTrain2)
resTest2 = np.array(resTest2)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7), sharey=True)
ax1.set_title('Accuracy en fonction de knn')
ax2.set_title('Score f1 en fonction de knn')
# ax1.text(60, 0.90, "yellow:  xTest par moyenne MFCC\ngreen:   xTrain par moyenne MFCC\nred:       xTest par 1 MFCC\nblue:     xTrain par 1 MFCC")
# ax2.text(60, 0.90, "yellow:  xTest par moyenne MFCC\ngreen:   xTrain par moyenne MFCC\nred:       xTest par 1 MFCC\nblue:     xTrain par 1 MFCC")
ax1.plot(resInit, resTrain2[:, 0], color="g", label="xTrain par moyenne MFCC")
ax1.plot(resInit, resTest2[:, 0], color="y", label="xTest par moyenne MFCC")
ax1.legend()
ax2.legend()
plt.show()


**Question 4:** Parmi les 2 représentations proposées, laquelle vous parait la meilleure?

Réponse : Après plusieurs essais, la méthode de la moyenne me semble meilleur

# TP Standardisation

## Partie 1: standardisation

Comparez les métriques de classification par kNN soit avec les représentations par moyenne des MFCCs soit par ces représentations après standardisation.
Attention: pour la standardisation, les moyennes et écarts types doivent être calculés sur les données d'apprentissage (train). C'est la même standardisation qui doit être appliquée à toutes les données (train/valid/test).

## Partie 2: PCA

Implémenter l'algorithme de PCA en utilisant le guide dans les derniers slides du cours. Attention, la transformation s'apprend sur les données d'apprentissage (train) et s'applique ensuite sur toutes les données.

Utilisez la PCA pour visualiser les données en 2D.

In [ ]:
# loading train dataset
nb = 500  # for each class
i = 0
clNumber = 0
yTrain = []
xTrain = np.empty(shape=(nb * nbClasses, 13), dtype=float)
for cl in classes:
    listSons = glob.glob(basedir + cl + "/*.wav")
    for s in listSons[:nb]:
        (sig, rate) = librosa.load(s)
        mfcc_feat = librosa.feature.mfcc(y=sig, sr=rate, n_mfcc=13)
        xTrain[i] = np.mean(mfcc_feat, axis=1)
        i += 1
    yTrain += [clNumber] * nb
    clNumber += 1

yTrain = np.array(yTrain)

In [ ]:
## loading test dataset
i = 0
clNumber = 0
yTest = []
xTest = np.empty(shape=(nb * nbClasses, 13), dtype=float)
for cl in classes:
    listSons = glob.glob(basedir + cl + "/*.wav")
    for s in listSons[-nb:]:
        (sig, rate) = librosa.load(s)
        mfcc_feat = librosa.feature.mfcc(y=sig, sr=rate, n_mfcc=13)
        xTest[i] = np.mean(mfcc_feat, axis=1)
        i += 1
    yTest += [clNumber] * nb
    clNumber += 1

yTest = np.array(yTest)

In [ ]:
# loading valid dataset
i = 0
clNumber = 0
yValid = []
xValid = np.empty(shape=(nb * nbClasses, 13), dtype=float)
for cl in classes:
    listSons = glob.glob(basedir + cl + "/*.wav")
    for s in listSons[nb:nb*2]:
        (sig, rate) = librosa.load(s)
        mfcc_feat = librosa.feature.mfcc(y=sig, sr=rate, n_mfcc=13)
        xValid[i] = np.mean(mfcc_feat, axis=1)
        i += 1
    yValid += [clNumber] * nb
    clNumber += 1

yValid = np.array(yValid)

In [ ]:
n = xTrain.shape[0]
mu = np.mean(xTrain, axis=0)
var = np.sum((xTrain - mu)**2, axis=0) / n
sigma = np.sqrt(var)
mad = np.sum(np.abs(xTrain - mu), axis=0) / n

print(n)
print(mu)
print(var)
print(sigma)
print(mad)

In [ ]:
xTrainNorm = (xTrain - mu) / sigma
xTrainStand = (xTrain - mu) / mad
xTestNorm = (xTest - mu) / sigma
xTestStand = (xTest - mu) / mad
xValidNorm = (xValid - mu) / sigma
xValidStand = (xValid - mu) / mad

resTrain = []
resTrainNorm = []
resTrainStand = []
resTest = []
resTestNorm = []
resTestStand = []
resValid = []
resValidNorm = []
resValidStand = []

for i in range(1, 200):
    resTrain.append(metriques(xTrain, yTrain, xTrain, i, classes, nb))
    resTrainNorm.append(metriques(xTrainNorm, yTrain, xTrainNorm, i, classes, nb))
    resTrainStand.append(metriques(xTrainStand, yTrain, xTrainStand, i, classes, nb))
    resTest.append(metriques(xTrain, yTrain, xTest, i, classes, nb))
    resTestNorm.append(metriques(xTrainNorm, yTrain, xTestNorm, i, classes, nb))
    resTestStand.append(metriques(xTrainStand, yTrain, xTestStand, i, classes, nb))
    resValid.append(metriques(xTrain, yTrain, xValid, i, classes, nb))
    resValidNorm.append(metriques(xTrainNorm, yTrain, xValidNorm, i, classes, nb))
    resValidStand.append(metriques(xTrainStand, yTrain, xValidStand, i, classes, nb))

resInit = [i for i in range(1, 200)]

resTrain = np.array(resTrain)
resTrainNorm = np.array(resTrainNorm)
resTrainStand = np.array(resTrainStand)
resTest = np.array(resTest)
resTestNorm = np.array(resTestNorm)
resTestStand = np.array(resTestStand)
resValid = np.array(resValid)
resValidNorm = np.array(resValidNorm)
resValidStand = np.array(resValidStand)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 7), sharey=True)
ax1.set_title("Accuracy en fonction de knn avec Train")
ax2.set_title("Accuracy en fonction de knn avec Test")
ax3.set_title("Accuracy en fonction de knn avec Valid")
ax1.plot(resInit, resTrain[:, 0], color="b", label="xTrain dans xTrain")
ax1.plot(resInit, resTrainNorm[:, 0], color="g", label="xTrainNorm dans xTrainNorm")
ax1.plot(resInit, resTrainStand[:, 0], color='r', label="xTrainStand dans xTrainStand")
ax2.plot(resInit, resTest[:, 0], color="b", label="xTest dans xTrain")
ax2.plot(resInit, resTestNorm[:, 0], color="g", label="xTestNorm dans xTrainNorm")
ax2.plot(resInit, resTestStand[:, 0], color='r', label="xTestStand dans xTrainStand")
ax3.plot(resInit, resValid[:, 0], color="b", label="xValid dans xTrain")
ax3.plot(resInit, resValidNorm[:, 0], color="g", label="xValidNorm dans xTrainNorm")
ax3.plot(resInit, resValidStand[:, 0], color='r', label="xValidStand dans xTrainStand")
ax1.legend()
ax2.legend()
ax3.legend()
plt.show()